In [29]:
import pandas as pd
import numpy as np

import os

I'll start with reading data. I will unzip and convert it to .csv file

In [ ]:
import os
from zipfile import ZipFile

# path to my folder
zip_folder = "C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\october2023typeB"
output_folder = "C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\Unziped_Data"

# make sure that output folder exists
os.makedirs(output_folder, exist_ok=True)

# work with all zip files in folder
for zip_filename in os.listdir(zip_folder):
    if zip_filename.endswith(".zip"):
        with ZipFile(os.path.join(zip_folder, zip_filename), 'r') as zip_ref:
            # Extract all files to the output folder
            zip_ref.extractall(output_folder)

#.txt to .csv
for file in os.listdir(output_folder):
    if file.endswith(".txt"):
        txt_path = os.path.join(output_folder, file)
        csv_path = txt_path.replace(".txt", ".csv")
        os.rename(txt_path, csv_path)

print("Unzipping and renaming complete!")


Unzipping and renaming complete!


Now my goal is to combine all this .csv files in one dataframe. I will creat new column for each row named `date`, where I will save date of operation.<br>I will save only data with `AFKS, AFLT, AGRO, ALRS, CBOM, GAZP, GLTR, GMKN, MAGN, MGNT, SBER, SBERP, TRNFP, VKCO, VTBR` constituents cause only them I will describe in this assignment.<br> At the same time I will remove all data that include `NaN` in price column


In [31]:
import pandas as pd
import os

# Output file path
output_folder = "C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\Unziped_Data"

save_to = "C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\combined_data\\combined_order_logs.csv"

# List of SECCODES to keep
seccodes = ["AFKS", "AFLT", "AGRO", "ALRS", "CBOM", "GAZP", "GLTR", "GMKN", 
                  "MAGN", "MGNT", "SBER", "SBERP", "TRNFP", "VKCO", "VTBR"]

# list of all .csv files in the folder
files = [f for f in os.listdir(output_folder) if f.endswith('.csv')]

for i, file in enumerate(files):
    
    date = file[-12:-4]
    
    # Read file in chunks
    chunks = pd.read_csv(os.path.join(output_folder, file), chunksize=100000)
    
    for chunk in chunks:
        # I'll filter based on SECCODES
        filtered_chunk = chunk[chunk['SECCODE'].isin(seccodes)]
        
        # Drop rows with NaN values in any column
        cleaned_chunk = filtered_chunk.dropna(subset=['PRICE'])
        
        
        cleaned_chunk['date'] = date
        
        # Append or create the combined file
        mode = 'w' if i == 0 and chunk.index[0] == 0 else 'a'  # write for the 1st chunk, append for others
        header = i == 0 and chunk.index[0] == 0  # write header only for the 1st chunk
        cleaned_chunk.to_csv(save_to, mode=mode, header=header, index=False)

print(f"\n\n\nFiltered and cleaned data from all files combined and saved to: combined_order_logs.csv")

# Load the entire combined file into a DataFrame
combined_data = pd.read_csv(save_to, index_col=False)
print("Combined data loaded into DataFrame successfully.")




Filtered and cleaned data from all files combined and saved to: combined_order_logs.csv
Combined data loaded into DataFrame successfully.


In [4]:
combined_data = pd.read_csv("C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\combined_data\\combined_order_logs.csv", index_col=False)
print(f"Size of my dataframe: {combined_data.shape}")

combined_data.head(10)

Size of my dataframe: (68322575, 7)


,SECCODE,BUYSELL,TIME,TRADENO,PRICE,VOLUME,date
0,GAZP,B,100000005313,8.521259e+09,167.83,10,20231002
1,GAZP,B,100000005313,NaN,179.42,10,20231002
2,GAZP,S,100000005313,NaN,151.00,2570,20231002
3,GAZP,S,100000005313,8.521259e+09,167.83,10,20231002
4,GAZP,B,100000005313,8.521259e+09,167.83,10,20231002
5,GAZP,S,100000005313,8.521259e+09,167.83,10,20231002
6,GAZP,B,100000005313,8.521259e+09,167.83,50,20231002
7,GAZP,S,100000005313,8.521259e+09,167.83,50,20231002
8,GAZP,B,100000005313,8.521259e+09,167.83,10,20231002
9,GAZP,S,100000005313,8.521259e+09,167.83,10,20231002


In the end I can say that we've stored data only with constituents that we need and without any Nan values in price column and save it to `combined_order_logs.csv`. <br> Now we have: 68322575 numbers of rows in our dataframe. Lets see how we can clean our data more<br>We can remove all rows were TRADENO - Nan cause it's number of сделки

In [5]:
filtered_data = combined_data.dropna(subset=['TRADENO'])

print(f"We've done. Size of new dataframe: {filtered_data.shape}\n\nWe've removed {100 - ((filtered_data.shape[0] / combined_data.shape[0]) * 100):.3f}% of data for TRADENO criteria")

We've done. Size of new dataframe: (18317572, 7)

We've removed 73.190% of data for TRADENO criteria


In [6]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18317572 entries, 0 to 68322574
Data columns (total 7 columns):
 #   Column   Dtype  
---  ------   -----  
 0   SECCODE  object 
 1   BUYSELL  object 
 2   TIME     int64  
 3   TRADENO  float64
 4   PRICE    float64
 5   VOLUME   int64  
 6   date     int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 1.1+ GB


As we've learned from pdf. We have Time in the next format: HHMMSSZZZXXX 

In [ ]:
#create hour and minute columns from the TIME column
filtered_data['hour'] = filtered_data['TIME'].astype(str).str[:2].astype(int) 
filtered_data['minute'] = filtered_data['TIME'].astype(str).str[2:4].astype(int)

# filter 9:50 - 18:50  
filtered_data_1 = filtered_data[
    ((filtered_data['hour'] == 9) & (filtered_data['minute'] >= 50)) |
    ((filtered_data['hour'] > 9) & (filtered_data['hour'] < 18)) |   
    ((filtered_data['hour'] == 18) & (filtered_data['minute'] <= 50)) 
]

# drop 'hour' and 'minute' columns
filtered_data_1 = filtered_data_1.drop(columns=['hour', 'minute'])

print(f"We've done. Size of new dataframe: {filtered_data_1.shape}\n\nWe've removed {100 - ((filtered_data_1.shape[0] / filtered_data.shape[0]) * 100):.3f}% of data for time criteria")


C:\Users\user\AppData\Local\Temp\ipykernel_4476\1625286355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['hour'] = filtered_data['TIME'].astype(str).str[:2].astype(int)
C:\Users\user\AppData\Local\Temp\ipykernel_4476\1625286355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['minute'] = filtered_data['TIME'].astype(str).str[2:4].astype(int)


We've done. Size of new dataframe: (18255996, 7)

We've removed 0.336% of data for time criteria


Let's
 remove all duplicates 


In [13]:
filtered_data_1 = filtered_data_1.drop_duplicates()

print(f"We've done. Size of new dataframe: {filtered_data_1.shape}")


We've done. Size of new dataframe: (18255996, 7)


As we can see, we don't have any duplicates<br>

In [22]:
column_names = ["SECCODE","BUYSELL","TIME","VOLUME","date"]

for name in column_names:
    nan_counts = filtered_data_1[name].isna().sum()
    print(f'{name}: {nan_counts}')

SECCODE: 0
BUYSELL: 0
TIME: 0
VOLUME: 0
date: 0


In [23]:
# let me cheack how many unique values do we have
# should be SECCODE = 15, BUYSELL = 2, date = 22
object_features = ['SECCODE', 'BUYSELL', 'date']

for column in object_features:
    print(f"{column}: {filtered_data_1[column].nunique()} unique values")

SECCODE: 15 unique values
BUYSELL: 2 unique values
date: 22 unique values


In [36]:
# remove rows where PRICE has = 0
preprocessed_df = filtered_data_1[filtered_data_1['PRICE'] != 0]
print(preprocessed_df.shape)

# Display the shape of the resulting DataFrame
print(f"We've done. Size of new dataframe: {preprocessed_df.shape}")

(18255996, 7)
We've done. Size of new dataframe: (18255996, 7)


And finally we can remove duplicate trades where the same transaction is recorded twice: once as `Buy` and once as `Sell`

In [37]:
processed_df = preprocessed_df.drop_duplicates(subset=['SECCODE', 'TIME', 'TRADENO', 'PRICE', 'VOLUME', 'date'])

print(f"We've done. Size of new dataframe: {processed_df.shape}\n\nWe've removed {(((preprocessed_df.shape[0] - processed_df.shape[0])/ preprocessed_df.shape[0]) * 100):.3f}% of data for `transaction duplicates` criteria")


# Save the cleaned DataFrame to a new CSV file if needed
processed_df.to_csv("C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\Processed_data\\processed_data.csv", index=False)

We've done. Size of new dataframe: (9127998, 7)

We've removed 50.000% of data for `transaction duplicates` criteria


Now we've done with preprocessing of data. 
### Next step is to:
1. Calculate average daily ruble volume
2. Daily realized volatility (the square root of the sum of all squared log-differences of prices during the day)
3. Number of transactions
4. Median ruble size of transactions

In [38]:
data = pd.read_csv("C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\Processed_data\\processed_data.csv")

# it's our seccode
seccodes = data['SECCODE'].unique()

# dictionary to store results
results = {}

# Loop through each SECCODE
for code in seccodes:
    # filter data
    stock_data = data[data['SECCODE'] == code]
    
    # group by date for daily calculations
    grouped = stock_data.groupby('date')
    
    # average daily ruble volume
    buy_avg_daily_volume = grouped.apply(lambda x: (x['PRICE'] * x['VOLUME']).sum()).mean()
    
    # daily realized volatility
    def realized_volatility(group):
        log_returns = np.log(group['PRICE']).diff()
        return np.sqrt((log_returns ** 2).sum())
    
    buy_volatility = grouped.apply(realized_volatility).mean()
    
    # number of transactions
    buy_transactions = grouped.size().mean()
    
    # median ruble size of transactions
    buy_median_ruble_size = grouped.apply(lambda x: (x['PRICE'] * x['VOLUME']).median()).median()
    
    # Store results for this SECCODE
    results[code] = {
        "Average Daily Volume": buy_avg_daily_volume,
        "Daily Volatility": buy_volatility,
        "Average Transactions": buy_transactions,
        "Median Ruble Size": buy_median_ruble_size,
    }

results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)

# Save results to a CSV file
results_df.to_csv("4th_task.csv")


C:\Users\user\AppData\Local\Temp\ipykernel_4476\4143237582.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  buy_avg_daily_volume = grouped.apply(lambda x: (x['PRICE'] * x['VOLUME']).sum()).mean()
C:\Users\user\AppData\Local\Temp\ipykernel_4476\4143237582.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  buy_volatility = grouped.apply(realized_volatility).mean()
C:\Users\user\AppData\Local\Temp\ipyker

       Average Daily Volume  Daily Volatility  Average Transactions  \
GAZP           3.527903e+09          0.015113          57816.727273   
AGRO           1.328405e+09          0.042442          31938.409091   
ALRS           1.155141e+09          0.027572          35250.636364   
AFLT           2.762012e+08          0.025703          10220.863636   
SBER           8.199113e+09          0.012620          87203.000000   
AFKS           3.307517e+08          0.016860           9987.500000   
GMKN           1.511901e+09          0.016184          16858.772727   
MGNT           1.749113e+09          0.019400          22706.681818   
MAGN           5.997891e+08          0.021033          22077.863636   
SBERP          6.654762e+08          0.013609          12259.090909   
VTBR           2.038567e+09          0.031034          56356.000000   
TRNFP          1.685541e+09          0.016451           5432.636364   
GLTR           1.252516e+08          0.025501           9372.409091   
CBOM  

C:\Users\user\AppData\Local\Temp\ipykernel_4476\4143237582.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  buy_avg_daily_volume = grouped.apply(lambda x: (x['PRICE'] * x['VOLUME']).sum()).mean()
C:\Users\user\AppData\Local\Temp\ipykernel_4476\4143237582.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  buy_volatility = grouped.apply(realized_volatility).mean()
C:\Users\user\AppData\Local\Temp\ipyker

We've done with this task. Result u can see in `4th_task.csv`


Answer the following questions:
1. Which stock was the most/least volatile during the period under investigation?
2. Which stock had the largest/lowest share volume? ruble volume?
3. Which stock had the largest/lowest number of transactions?
4. Which stock had the largest/lowest number of quotes changes?
5. On which day of the week stocks had the largest/lowest volume in the sample?
6. Which stock had the highest/lowest ruble bid-ask spread? percentage bid-ask
spread?

In [ ]:
try:
    metrics = pd.read_csv("4th_task.csv")
except FileNotFoundError:
    print("Metrics file not found. Recalculating metrics from the dataset.")
    metrics = None

data = pd.read_csv("C:\\Users\\user\\Desktop\\Innopolis_courses\\Financy_project\\data\\Processed_data\\processed_data.csv") 

# adding weekday column for weekly calculations
data['weekday'] = pd.to_datetime(data['date'], format='%Y%m%d').dt.day_name()

# function to calculate realized volatility
def realized_volatility(group):
    log_returns = np.log(group['PRICE']).diff()
    return np.sqrt((log_returns ** 2).sum())

# most/least volatile stock
if metrics is not None and 'Realized Volatility' in metrics.columns:
    most_volatile = metrics.loc[metrics['Realized Volatility'].idxmax(), 'SECCODE']
    least_volatile = metrics.loc[metrics['Realized Volatility'].idxmin(), 'SECCODE']
else:
    volatility = data.groupby('SECCODE').apply(realized_volatility)
    most_volatile = volatility.idxmax()
    least_volatile = volatility.idxmin()

# largest/lowest share volume and ruble volume
if metrics is not None and 'Combined Avg Daily Volume' in metrics.columns:
    largest_share_volume = metrics.loc[metrics['Combined Avg Daily Volume'].idxmax(), 'SECCODE']
    lowest_share_volume = metrics.loc[metrics['Combined Avg Daily Volume'].idxmin(), 'SECCODE']
    # with using ruble volume metrics
    largest_ruble_volume = metrics.loc[metrics['Buy Avg Daily Volume'].idxmax(), 'SECCODE']
    lowest_ruble_volume = metrics.loc[metrics['Sell Avg Daily Volume'].idxmin(), 'SECCODE']
else:
    share_volume = data.groupby('SECCODE')['VOLUME'].sum()
    ruble_volume = data.groupby('SECCODE').apply(lambda x: (x['PRICE'] * x['VOLUME']).sum())
    largest_share_volume = share_volume.idxmax()
    lowest_share_volume = share_volume.idxmin()
    largest_ruble_volume = ruble_volume.idxmax()
    lowest_ruble_volume = ruble_volume.idxmin()

# largest/lowest number of transactions ---
if metrics is not None and 'Combined Avg Transactions' in metrics.columns:
    largest_transactions = metrics.loc[metrics['Combined Avg Transactions'].idxmax(), 'SECCODE']
    lowest_transactions = metrics.loc[metrics['Combined Avg Transactions'].idxmin(), 'SECCODE']
else:
    transactions = data.groupby('SECCODE').size()
    largest_transactions = transactions.idxmax()
    lowest_transactions = transactions.idxmin()

# largest/lowest number of quote changes
quote_changes = data.groupby('SECCODE')['PRICE'].nunique()
largest_quote_changes = quote_changes.idxmax()
lowest_quote_changes = quote_changes.idxmin()

#day of the week with largest/lowest volume
weekday_grouped = data.groupby('weekday')
weekday_ruble_volume = weekday_grouped.apply(lambda x: (x['PRICE'] * x['VOLUME']).sum())
largest_volume_day = weekday_ruble_volume.idxmax()
lowest_volume_day = weekday_ruble_volume.idxmin()

# highest/lowest ruble bid-ask spread and percentage spread
if 'ASK' in data.columns and 'BID' in data.columns:
    data['ruble_spread'] = data['ASK'] - data['BID']
    data['percentage_spread'] = (data['ASK'] - data['BID']) / ((data['ASK'] + data['BID']) / 2) * 100
    spread_grouped = data.groupby('SECCODE')
    highest_ruble_spread = spread_grouped['ruble_spread'].mean().idxmax()
    lowest_ruble_spread = spread_grouped['ruble_spread'].mean().idxmin()
    highest_percentage_spread = spread_grouped['percentage_spread'].mean().idxmax()
    lowest_percentage_spread = spread_grouped['percentage_spread'].mean().idxmin()
else:
    highest_ruble_spread = None
    lowest_ruble_spread = None
    highest_percentage_spread = None
    lowest_percentage_spread = None


print(f"1. Most volatile stock: {most_volatile}, Least volatile stock: {least_volatile}")
print(f"2. Largest share volume: {largest_share_volume}, Lowest share volume: {lowest_share_volume}")
print(f"   Largest ruble volume: {largest_ruble_volume}, Lowest ruble volume: {lowest_ruble_volume}")
print(f"3. Largest transactions: {largest_transactions}, Lowest transactions: {lowest_transactions}")
print(f"4. Largest quote changes: {largest_quote_changes}, Lowest quote changes: {lowest_quote_changes}")
print(f"5. Day with largest volume: {largest_volume_day}, Day with lowest volume: {lowest_volume_day}")
if 'ASK' in data.columns and 'BID' in data.columns:
    print(f"6. Highest ruble spread: {highest_ruble_spread}, Lowest ruble spread: {lowest_ruble_spread}")
    print(f"   Highest percentage spread: {highest_percentage_spread}, Lowest percentage spread: {lowest_percentage_spread}")
else:
    print("6. ASK and BID columns not found. Cannot calculate spreads.")


C:\Users\user\AppData\Local\Temp\ipykernel_4476\574761241.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  volatility = data.groupby('SECCODE').apply(realized_volatility)
C:\Users\user\AppData\Local\Temp\ipykernel_4476\574761241.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ruble_volume = data.groupby('SECCODE').apply(lambda x: (x['PRICE'] * x['VOLUME']).sum())


1. Most volatile stock: VKCO, Least volatile stock: SBER
2. Largest share volume: VTBR, Lowest share volume: TRNFP
   Largest ruble volume: SBER, Lowest ruble volume: GLTR
3. Largest transactions: SBER, Lowest transactions: TRNFP
4. Largest quote changes: AGRO, Lowest quote changes: TRNFP
5. Day with largest volume: Monday, Day with lowest volume: Thursday
6. ASK and BID columns not found. Cannot calculate spreads.


C:\Users\user\AppData\Local\Temp\ipykernel_4476\574761241.py:58: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weekday_ruble_volume = weekday_grouped.apply(lambda x: (x['PRICE'] * x['VOLUME']).sum())


## Next task:<br>  
Examine empirically what is the relationship between the number of trades and the
number of quotes changes? For example, plot the log of the number of trades against
the log of the number of quotes changes calculated for each stock and each trading day.
